# BERT fine-tune for Persuasive Essays dataset

BERT fine-tune for the Persuasive Essays dataset with our 'combined contextual and structural features as text' sentence representation.

## Packages

In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## Imports

In [2]:
import os
import pickle

from collections import Counter

from sklearn.metrics import classification_report

import numpy as np
import torch
import torch.nn as nn

import transformers
from transformers import Trainer
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers.data.data_collator import DataCollatorWithPadding

import datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import load_metric

## File and folder paths

In [3]:
# Your paths go here:

DATA_FILE = '../datasets/dataset_persuasive_essays_icann.pt'
RESULTS_FOLDER = '../results/'

## GPU

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda')

## Load dataset

In [6]:
dataset = torch.load(DATA_FILE)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['split', 'text', 'labels', 'sentence', 'topic_and_full_sentence', 'topic_full_sentence_stuctural_fts', 'topic_full_sentence_structural_fts_combined', 'feature_tensor'],
        num_rows: 4709
    })
    test: Dataset({
        features: ['split', 'text', 'labels', 'sentence', 'topic_and_full_sentence', 'topic_full_sentence_stuctural_fts', 'topic_full_sentence_structural_fts_combined', 'feature_tensor'],
        num_rows: 1258
    })
})

## Tokenizer

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [10]:
label_names = set(dataset['train']['labels'])
label_nb = len(label_names)
labels = ClassLabel(num_classes=label_nb, names=label_names)

In [11]:
labels

ClassLabel(num_classes=3, names={'Premise', 'Claim', 'MajorClaim'}, names_file=None, id=None)

To obtain the different results with different sentence representation, change the column name in the cell below. The column names correspond to the following sentence representations:
1. sentence representation: component only, column_name = text
2. sentence representation: sentence, column_name = sentence
3. sentence representation: topic + sent, column_name = topic_and_full_sentence
4. sentence representation: topic + sent + struct fts, column_name = topic_full_sentence_stuctural_fts
5. sentence representation: topic + sent + struct fts (abbreviated), column_name = topic_full_sentence_structural_fts_combined

In [12]:
column_name = 'topic_full_sentence_structural_fts_combined'
def tokenize(batch):
    tokens = tokenizer(batch[column_name], truncation=True, padding=True, max_length=512)
    tokens['labels'] = labels.str2int(batch['labels'])
    return tokens

In [13]:
dataset = dataset.map(tokenize, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [14]:
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'feature_tensor', 'input_ids', 'labels', 'sentence', 'split', 'text', 'token_type_ids', 'topic_and_full_sentence', 'topic_full_sentence_structural_fts_combined', 'topic_full_sentence_stuctural_fts'],
        num_rows: 4709
    })
    test: Dataset({
        features: ['attention_mask', 'feature_tensor', 'input_ids', 'labels', 'sentence', 'split', 'text', 'token_type_ids', 'topic_and_full_sentence', 'topic_full_sentence_structural_fts_combined', 'topic_full_sentence_stuctural_fts'],
        num_rows: 1258
    })
})

## Train-test split

In [16]:
train_dataset = dataset['train'].shuffle(seed=42)
test_dataset = dataset['test'].shuffle(seed=42)

train_val_datasets = dataset['train'].train_test_split(train_size=0.8, seed=42)
train_dataset = train_val_datasets['train']
val_dataset = train_val_datasets['test']

In [17]:
dataset_d = {}
dataset_d['train'] = train_dataset
dataset_d['test'] = test_dataset
dataset_d['val'] = val_dataset

## Model

In [22]:
NUM_LABELS = labels.num_classes
BATCH_SIZE = 48
NB_EPOCHS = 6

In [23]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=NUM_LABELS)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## Trainer

In [24]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.CrossEntropyLoss()#(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [25]:
metric = load_metric('f1')

def compute_metrics(eval_pred):
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    return metric.compute(predictions=predictions, references=labels, average='macro')

In [26]:
training_args = TrainingArguments(
    
    # output
    output_dir=RESULTS_FOLDER,          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=1e-5,#2e-5,                 # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
    # eval
    evaluation_strategy="steps",              # cf. paper Sun et al.
    eval_steps=20,                            # cf. paper Sun et al.
    
    # log
    logging_dir="/notebooks/Results/bert_sequence_classification/tb_logs",  
    logging_strategy='steps',
    logging_steps=20,
    
    # save
    save_strategy='steps',
    save_total_limit=2,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    # metric_for_best_model='eval_loss' 
    metric_for_best_model='f1'
)

In [27]:
trainer = CustomTrainer( # Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [28]:
results = trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: split, text, topic_and_full_sentence, feature_tensor, topic_full_sentence_structural_fts_combined, topic_full_sentence_stuctural_fts, sentence.
***** Running training *****
  Num examples = 3767
  Num Epochs = 6
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 1
  Total optimization steps = 474


Step,Training Loss,Validation Loss,F1
20,0.969800,0.909361,0.257279
40,0.857200,0.768948,0.257279
60,0.766600,0.699002,0.257279
80,0.662500,0.584569,0.447821
100,0.580700,0.511761,0.726736
120,0.536200,0.490413,0.742227
140,0.495700,0.478053,0.683323
160,0.497500,0.461664,0.752469
180,0.460600,0.486350,0.663104
200,0.440400,0.436899,0.775800


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: split, text, topic_and_full_sentence, feature_tensor, topic_full_sentence_structural_fts_combined, topic_full_sentence_stuctural_fts, sentence.
***** Running Evaluation *****
  Num examples = 942
  Batch size = 48
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: split, text, topic_and_full_sentence, feature_tensor, topic_full_sentence_structural_fts_combined, topic_full_sentence_stuctural_fts, sentence.
***** Running Evaluation *****
  Num examples = 942
  Batch size = 48
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: split, text, topic_and_full_sentence, feature_tensor, topic_full_sentence_structural_fts_combined, topic_full_sentence_stuctural_ft

## Results

In [29]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [30]:
test_trainer = Trainer(model, data_collator=DataCollatorWithPadding(tokenizer))
test_raw_preds, test_labels, _ = test_trainer.predict(test_dataset)
test_preds = np.argmax(test_raw_preds, axis=1)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: split, text, topic_and_full_sentence, feature_tensor, topic_full_sentence_structural_fts_combined, topic_full_sentence_stuctural_fts, sentence.
***** Running Prediction *****
  Num examples = 1258
  Batch size = 8


In [32]:
target_name = labels.int2str([0,1,2])
print(classification_report(test_labels, test_preds, target_names=target_name))

              precision    recall  f1-score   support

     Premise       0.92      0.89      0.91       805
       Claim       0.69      0.67      0.68       301
  MajorClaim       0.79      0.95      0.87       152

    accuracy                           0.85      1258
   macro avg       0.80      0.84      0.82      1258
weighted avg       0.85      0.85      0.85      1258

